### `Conv2DPytTestGroups` code

In [281]:
import numpy as np

In [282]:
class Conv2DPytTestGroups:
    ''' Computes convolution given the input parameters '''
    
    ''' * The class implementation will be along the lines of torch.nn.Conv2D in order to 
          enable comparison of this NumPy only implementation and seamless testing
        * Can expect extensive refactoring of the existing code in the days to come
        * As part of refactoring, some code will be de-modularized
        * Old code will be retained at the end of the notebook for reference
    '''
    '''
        TODO:
        * Fix `groups` implementation to make it more robust - some tests failing
        * Implementing other features and caveats offered by nn.torch.Conv2D 
          (e.g., uniform sampling of kernel weights etc.)
        * Optimizing code
    '''
    
    def __init__(
        self, 
        in_channels, 
        out_channels, 
        kernel_size, 
        padding = 0, 
        stride = 1, 
        dilation = 1, 
        groups = 1, 
        bias = True, 
        padding_mode = 'zeros', 
        device = None, 
        dtype = None, 
        verbose = True, 
        debug = False
        ):
        super(Conv2DPytTestGroups, self).__init__()
        
        ''' mandatory parameters '''
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        
        ''' optional parameters '''
        self.padding = padding
        self.stride = stride
        self.dilation = dilation
        self.groups = groups
        
        ''' optional parameters (dummy, yet to be implemented)'''
        self.bias = bias
        self.padding_mode = padding_mode
        self.device = device
        self.dtype = dtype
        
        ''' additional parameters (different from torch.nn.Conv2D)'''
        self.verbose = verbose
        self.verboseprint = print if self.verbose else lambda *a, **k: None
        self.debug = debug
        self.debugprint = print if self.debug else lambda *a, **k: None
        self.verboseprint('*** parameters ***')
        self.verboseprint('in_channels: {}, out_channels: {}, kernel_size: {}'.format(self.in_channels, self.out_channels, self.kernel_size))
        self.verboseprint('padding: {}, stride: {}, dilation factor: {}'.format(self.padding, self.stride, self.dilation))
        self.verboseprint('groups: {}, bias: {}, padding_mode: {}, device: {}, dtype: {}'.format(self.groups, self.bias, self.padding_mode, self.device, self.dtype))
        self.verboseprint('\n')
    
    def forward(self, _input, kernels):
        ''' forward pass to perform convolution '''
        
        ''' do error checking '''
        _input_n, _input_c, _input_h, _input_w = _input.shape
        if _input_h + 2 * self.padding < self.dilation * (self.kernel_size[0] - 1) + 1: # check if (dilated) ker_h is valid
            raise Exception('invalid input parameters: kernel height is larger than input height')
        if _input_w + 2 * self.padding < self.dilation * (self.kernel_size[1] - 1) + 1: # check if (dilated) ker_w is valid
            raise Exception('invalid input parameters: kernel width is larger than input width')
        if ((_input_h + 2 * self.padding - (self.dilation * (self.kernel_size[0] - 1) + 1)) / self.stride) + 1 < 0: # check if out_h is valid
            raise Exception('invalid input parameters: output height is negative')
        if ((_input_h + 2 * self.padding - (self.dilation * (self.kernel_size[1] - 1) + 1)) / self.stride) + 1 < 0: # check if out_w is valid
            raise Exception('invalid input parameters: output width is negative')
        if  self.in_channels % self.groups != 0: # check if groups is valid
            raise Exception('invalid input parameters: input channels is not divisible by groups')
        if self.out_channels % self.groups != 0: # check if groups is valid
            raise Exception('invalid input parametes: output channels is not divisible by groups')
         
        ''' add zero padding based on the input parameters '''
        if self.padding != 0:
            _input = np.array([[np.pad(channel,self.padding, 'constant', constant_values = 0) for channel in batch] for batch in _input])    
            self.verboseprint('*** padded input image ***')
            self.verboseprint('input batches: {}, input channels: {}, input height: {}, input weight: {}'.format(_input.shape[0], _input.shape[1], _input.shape[2], _input.shape[3]))
            self.verboseprint(_input)
            self.verboseprint('\n')
        if self.debug:
            for b in range(_input.shape[0]):
                _input[b] = (b+1) * np.ones_like(_input[b]) # define an image of all ones (twos etc.) based on the input parameters
        
        ''' use the provided kernels or create random kernels based on the input kernel parameters '''
        if kernels is not None:
            self.verboseprint('*** kernels ***')
            self.verboseprint('kernels: {}, kernel channels: {}, kernel height: {}, kernel weight: {}'.format(self.out_channels, int(self.in_channels / self.groups), self.kernel_size[0], self.kernel_size[1]))
        else:
            kernels = []
            self.verboseprint('*** kernels ***')
            self.verboseprint('kernels: {}, kernel channels: {}, kernel height: {}, kernel weight: {}'.format(self.out_channels, int(self.in_channels / self.groups), self.kernel_size[0], self.kernel_size[1]))
            for k in range(self.out_channels):
                kernel = np.random.rand(int(self.in_channels/self.groups), self.kernel_size[0], self.kernel_size[1]) # define a random kernel based on the kernel parameters
                if self.debug:
                    kernel = k * np.ones_like(kernel)
                kernels.append(kernel)
                self.verboseprint('kernel {}'.format(k))
                self.verboseprint(kernel)
            self.verboseprint('\n')
        
        ''' dilate a kernel '''
        dil_ker_h = self.dilation * (self.kernel_size[0] - 1) + 1
        dil_ker_w = self.dilation * (self.kernel_size[1] - 1) + 1
        dil_kernels = []
        for kernel in kernels:
            dil_kernel = []
            for channel in kernel:
                dil_channel = np.zeros((dil_ker_h, dil_ker_w))
                for row in range(len(channel)):
                    for col in range(len(channel[0])):
                        dil_channel[self.dilation*row][self.dilation*col] = channel[row][col]
                dil_kernel.append(dil_channel.tolist())
            dil_kernels.append(dil_kernel)
        kernels, self.kernel_size = dil_kernels, (dil_ker_h, dil_ker_w)
        self.verboseprint('*** dilated kernels ***')
        self.verboseprint('kernels: {}, dilation factor: {}, kernel channels: {}, kernel height: {}, kernel weight: {}'.format(self.out_channels, self.dilation, int(self.in_channels / self.groups), self.kernel_size[0], self.kernel_size[1]))
        for k in range(self.out_channels):
            self.verboseprint('kernel {}'.format(k))
            self.verboseprint(kernels[k])
        self.verboseprint('\n')
        
        ''' compute output volume from the input and kernel parameters '''
        _input_n, _, _input_h, _input_w = _input.shape
        out_n = int(_input_n)
        out_c = int(self.out_channels)
        out_h = int((_input_h - self.kernel_size[0])/self.stride) + 1
        out_w = int((_input_w - self.kernel_size[1])/self.stride) + 1
        output = np.zeros([out_n, out_c, out_h, out_w])
        
        ''' parse through every element of the output and compute the convolution value for that element '''
        for b in range(out_n):
            for k in range(out_c):
                for h in range(out_h):
                    for w in range(out_w):
                        # convolve kernel over the input slices
                        self.debugprint('kernel indices, image indices')
                        self.debugprint('[c, h, w]', '[n, c, h, w]')
                        convol_sum = 0
                        ker_c = int(self.in_channels / self.groups)
                        ker_h = self.kernel_size[0]
                        ker_w = self.kernel_size[1]
                        for c_ker in range(ker_c):
                            for h_ker in range(ker_h):
                                for w_ker in range(ker_w):
                                    if self.groups == 1:
                                        self.debugprint([k, c_ker, h_ker, w_ker], [b, c_ker, h_ker + self.stride * h, w_ker + self.stride * w])
                                        convol_sum += kernels[k][c_ker][h_ker][w_ker] * _input[b][c_ker][h_ker + self.stride*h][w_ker + self.stride*w] # if groups == 1
                                    elif self.groups == self.in_channels:
                                        self.debugprint([k, c_ker, h_ker, w_ker], [b, k, h_ker + self.stride * h, w_ker + self.stride * w])
                                        convol_sum += kernels[k][c_ker][h_ker][w_ker] * _input[b][k][h_ker + self.stride * h][w_ker + self.stride * w] # if groups == in_channels
                                    else:
                                        self.debugprint([k, c_ker, h_ker, w_ker], [b, c_ker if k < self.groups else c_ker + ker_c, h_ker + self.stride * h, w_ker + self.stride * w])
                                        convol_sum += kernels[k][c_ker][h_ker][w_ker] * _input[b][c_ker if k < self.groups else c_ker + ker_c][h_ker + self.stride * h][w_ker + self.stride * w] # if 1 < groups < in_channels
                        self.debugprint('\n')
                        output[b, k, h, w] += convol_sum
        self.verboseprint('*** Conv2DPytTest output ***')
        output_shape = output.shape
        self.verboseprint('output batches: {}, ouput channels: {}, output height: {}, output weight: {}'.format(output_shape[0], output_shape[1], output_shape[2], output_shape[3]))
        assert((out_n, out_c, out_h, out_w) == output_shape)
        self.verboseprint(output)
        self.verboseprint('\n')
        return output

### Standalone test (random kernel, random input)

In [283]:
debug = False # DO NOT CHANGE THIS while using Conv2DPytTestGroups - TODO: modify this flag to `test`

in_channels = 6 # input channels
out_channels = 4 # output channels
kernel_size = (2, 2) # kernel size

padding = 1 # padding (optional)
stride = 2 # stride (optional)
dilation = 1 # dilation factor (optional)
groups = 2 # groups (optional)

in_batches = 2 # input batches
in_h = 4 # input height
in_w = 4 # input weight

_input = np.random.rand(in_batches, in_channels, in_h, in_w) # define a random image based on the input parameters
kernels = []
for k in range(out_channels):
    kernel = np.random.rand(int(in_channels / groups), kernel_size[0], kernel_size[1]) # define a random kernel based on the kernel parameters
    kernels.append(kernel)

In [284]:
# get Conv2DPytTestGroups output with the random inputs

conv2dpyttestgroups = Conv2DPytTestGroups(in_channels, out_channels, kernel_size, stride = stride, padding = padding, dilation = dilation, groups = groups, debug = debug, verbose = False) # call an instance of the class with the input parameters 
_output = conv2dpyttestgroups.forward(_input, kernels) # perform convolution
print("*** Conv2DPytTestGroups output ***")
print(_output)

*** Conv2DPytTestGroups output ***
[[[[0.70427929 2.82019084 0.40261863]
   [1.54349942 3.35092143 1.13886681]
   [0.15326395 1.31998229 0.81070694]]

  [[0.54893892 2.10956146 0.52147893]
   [2.04195267 3.94973766 1.2034504 ]
   [0.55683908 1.47607389 0.69651989]]

  [[0.62516699 1.18566361 0.4936451 ]
   [2.59164193 3.02877504 0.71158585]
   [0.76244125 1.96040352 0.3562567 ]]

  [[0.6692644  1.53056299 1.11834824]
   [1.32763088 2.89185683 1.74017708]
   [0.48735152 1.84209336 1.11125937]]]


 [[[0.6128883  1.14325889 0.70231473]
   [2.3061121  2.80073564 0.67622515]
   [0.48600849 1.42516353 1.03900796]]

  [[0.58537121 0.96336564 0.70591761]
   [1.7755101  2.7285069  0.59874787]
   [0.836532   2.13708475 0.83282811]]

  [[0.58247132 1.52525674 0.24810848]
   [1.56954752 2.13698187 0.86372561]
   [1.16611727 1.18447742 0.20073716]]

  [[0.67099181 1.32553127 0.49698438]
   [1.23806643 2.68759124 1.63231083]
   [0.37516915 1.49920869 0.6274384 ]]]]


In [285]:
# get PyTorch output with the same random inputs as above

import torch

x = torch.DoubleTensor(_input)
weights = torch.stack([torch.DoubleTensor(kernel) for kernel in kernels])
output = torch.nn.functional.conv2d(x, weights, stride = stride, padding = padding, groups = groups)
print("*** PyTorch output ***")
print(output)

*** PyTorch output ***
tensor([[[[0.7043, 2.8202, 0.4026],
          [1.5435, 3.3509, 1.1389],
          [0.1533, 1.3200, 0.8107]],

         [[0.5489, 2.1096, 0.5215],
          [2.0420, 3.9497, 1.2035],
          [0.5568, 1.4761, 0.6965]],

         [[0.6252, 1.1857, 0.4936],
          [2.5916, 3.0288, 0.7116],
          [0.7624, 1.9604, 0.3563]],

         [[0.6693, 1.5306, 1.1183],
          [1.3276, 2.8919, 1.7402],
          [0.4874, 1.8421, 1.1113]]],


        [[[0.6129, 1.1433, 0.7023],
          [2.3061, 2.8007, 0.6762],
          [0.4860, 1.4252, 1.0390]],

         [[0.5854, 0.9634, 0.7059],
          [1.7755, 2.7285, 0.5987],
          [0.8365, 2.1371, 0.8328]],

         [[0.5825, 1.5253, 0.2481],
          [1.5695, 2.1370, 0.8637],
          [1.1661, 1.1845, 0.2007]],

         [[0.6710, 1.3255, 0.4970],
          [1.2381, 2.6876, 1.6323],
          [0.3752, 1.4992, 0.6274]]]], dtype=torch.float64)


In [286]:
# compare outputs of conv-Numpy and PyTorch
print(torch.equal(torch.round(torch.DoubleTensor(_output)), torch.round(output))) # need to round the output due to precision difference

True


### Extensive tests (random kernel, random input)

In [289]:
def valid_params(num_tests):
    # generates `num_samples` number of valid input and kernel parameters 
    params_list = []
    sample_count = 0
    while sample_count < num_tests:
        in_channels = np.random.randint(20) + 1 # input channels
        out_channels = np.random.randint(in_channels) + 1 # output channels
        
        ker_h = np.random.randint(20) + 1
        ker_w = np.random.randint(20) + 1
        kernel_size = (ker_h, ker_w) # kernel size
        
        padding = np.random.randint(10) + 1 # padding (optional)
        stride = np.random.randint(5) + 1 # stride (optional)
        dilation = np.random.randint(10) + 1 # dilation factor (optional)
        groups = np.random.randint(in_channels) + 1 # groups (optional)
        
        in_batches = np.random.randint(5) + 1 # input batches
        in_h = np.random.randint(30) + 5 # input height
        in_w = np.random.randint(30) + 5 # input weight
    
        ker_h_flag, ker_w_flag, out_h_flag, out_w_flag, in_group_flag, out_group_flag = True, True, True, True, True, True
        
        if in_h + 2 * padding < dilation * (ker_h - 1) + 1: # check if (dilated) ker_h is valid
            ker_h_flag = False
        if in_w + 2 * padding < dilation * (ker_w - 1) + 1: # check if (dilated) ker_w is valid
            ker_w_flag = False
        if ((in_h + 2 * padding - (dilation * (ker_h - 1) + 1)) / stride) + 1 < 0: # check if out_h is valid
            out_h_flag = False
        if ((in_w + 2 * padding - (dilation * (ker_w - 1) + 1)) / stride) + 1 < 0: # check if out_w is valid
            out_w_flag = False
        if  in_channels % groups != 0: # check if groups is valid
            in_group_flag = False
        if out_channels % groups != 0: # check if groups is valid
            out_group_flag = False
        
        if ker_h_flag and ker_w_flag and out_h_flag and out_w_flag and in_group_flag and out_group_flag:
            params_list.append({'in_channels': in_channels, 'out_channels': out_channels, 'kernel_size': kernel_size,
                          'padding': padding, 'stride': stride, 'dilation': dilation, 'groups': groups, 'in_batches': in_batches,
                          'in_h': in_h, 'in_w': in_w})
            sample_count += 1
    return params_list

In [290]:
# for loop sweeping different input parameters and testing the outputs of Conv2DPytTestGroups and PyTorch
from tqdm import tqdm

num_tests = 100 # only test parameter to be varied
num_passed = 0
params_list = valid_params(num_tests)
print('Number of tests: {}\n\n'.format(len(params_list)))

for i, params in enumerate(tqdm(params_list)):
    print('Test: {}\nParams: {}'.format(i, params))
    debug = False # DO NOT CHANGE THIS while using Conv2DPytTest - TODO: modify this flag to `test`

    in_channels = params['in_channels'] # input channels
    out_channels = params['out_channels'] # output channels
    
    kernel_size = params['kernel_size'] # kernel size

    padding = params['padding'] # padding (optional)
    stride = params['stride'] # stride (optional)
    dilation = params['dilation'] # dilation factor (optional)
    groups = params['groups'] # groups (optional)
    
    in_batches = params['in_batches'] # input batches
    in_h = params['in_h'] # input height
    in_w = params['in_w'] # input weight
    
    _input = np.random.rand(in_batches, in_channels, in_h, in_w) # define a random image based on the input parameters
    kernels = []
    for k in range(out_channels):
        kernel = np.random.rand(int(in_channels / groups), kernel_size[0], kernel_size[1]) # define a random kernel based on the kernel parameters
        kernels.append(kernel)
    
    try:
        # get Conv2DPytTest output with the random inputs
        conv2dpyttestgroups = Conv2DPytTestGroups(in_channels, out_channels, kernel_size, stride = stride, padding = padding, dilation = dilation, groups = groups, debug = debug, verbose = False) # call an instance of the class with the input parameters 
        _output = conv2dpyttestgroups.forward(_input, kernels) # perform convolution

        # get PyTorch output with the same random inputs as above
        x = torch.DoubleTensor(_input)
        weights = torch.stack([torch.DoubleTensor(kernel) for kernel in kernels])
        output = torch.nn.functional.conv2d(x, weights, stride = stride, padding = padding, dilation = dilation, groups = groups)
        
    except Exception as e:
        print(e)
        pass
    
    # compare outputs of conv-Numpy and PyTorch
    result = torch.equal(torch.round(torch.DoubleTensor(_output)), torch.round(output)) # need to round the output due to precision difference
    print('Result: {}\n\n'.format(result))
    if result:
        num_passed += 1

print('{} out of {} ({}%) tests passed'.format(num_passed, num_tests, float(100 * num_passed / num_tests)))
    

Number of tests: 100




  0%|                                                   | 0/100 [00:00<?, ?it/s]

Test: 0
Params: {'in_channels': 1, 'out_channels': 1, 'kernel_size': (5, 10), 'padding': 3, 'stride': 1, 'dilation': 2, 'groups': 1, 'in_batches': 4, 'in_h': 7, 'in_w': 20}
invalid input parameters: output width is negative
Result: True


Test: 1
Params: {'in_channels': 3, 'out_channels': 3, 'kernel_size': (10, 18), 'padding': 3, 'stride': 2, 'dilation': 1, 'groups': 3, 'in_batches': 5, 'in_h': 15, 'in_w': 12}
Result: True


Test: 2
Params: {'in_channels': 1, 'out_channels': 1, 'kernel_size': (2, 1), 'padding': 1, 'stride': 1, 'dilation': 5, 'groups': 1, 'in_batches': 3, 'in_h': 16, 'in_w': 20}
Result: True


Test: 3
Params: {'in_channels': 13, 'out_channels': 13, 'kernel_size': (4, 6), 'padding': 6, 'stride': 5, 'dilation': 3, 'groups': 13, 'in_batches': 5, 'in_h': 19, 'in_w': 20}


  4%|█▋                                         | 4/100 [00:00<00:13,  6.94it/s]

Result: True


Test: 4
Params: {'in_channels': 3, 'out_channels': 1, 'kernel_size': (3, 11), 'padding': 10, 'stride': 3, 'dilation': 3, 'groups': 1, 'in_batches': 3, 'in_h': 34, 'in_w': 21}


...

Result: True


Test: 96
Params: {'in_channels': 1, 'out_channels': 1, 'kernel_size': (15, 6), 'padding': 2, 'stride': 4, 'dilation': 2, 'groups': 1, 'in_batches': 3, 'in_h': 26, 'in_w': 10}
Result: True


Test: 97
Params: {'in_channels': 15, 'out_channels': 12, 'kernel_size': (1, 12), 'padding': 7, 'stride': 5, 'dilation': 2, 'groups': 3, 'in_batches': 5, 'in_h': 19, 'in_w': 19}


 99%|█████████████████████████████████████████▌| 99/100 [01:33<00:00,  2.00it/s]

Result: False


Test: 98
Params: {'in_channels': 6, 'out_channels': 4, 'kernel_size': (1, 1), 'padding': 10, 'stride': 2, 'dilation': 9, 'groups': 1, 'in_batches': 4, 'in_h': 26, 'in_w': 22}
Result: True


Test: 99
Params: {'in_channels': 12, 'out_channels': 1, 'kernel_size': (13, 8), 'padding': 7, 'stride': 3, 'dilation': 1, 'groups': 1, 'in_batches': 4, 'in_h': 32, 'in_w': 29}


100%|█████████████████████████████████████████| 100/100 [01:35<00:00,  1.05it/s]

Result: True


86 out of 100 (86.0%) tests passed


### Modular code for methods in `forward()`

### Modular code for `forward()`

### Old `Conv2D` code  for reference